In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import sys
import tensorflow as tf
import time

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

In [ ]:
imdb = keras.datasets.imdb
vocab_size = 10000
index_from = 3

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=vocab_size, index_from=index_from)

In [ ]:
word_index = imdb.get_word_index()
print(len(word_index))
print(word_index)

In [ ]:
word_index = {k: (v + 3) for k, v in word_index.items()}

In [ ]:
word_index['<PAD>'] = 0
word_index['<START>'] = 1
word_index['<UNK>'] = 2
word_index['<END>'] = 3

reverse_word_index = dict(
    [(value, key) for key, value in word_index.items()]
)

def decode_review(text_ids):
    return ' '.join([reverse_word_index.get(word_id, "<UNK") for word_id in text_ids])

decode_review(train_data[0])

In [ ]:
max_length = 500

train_data = keras.preprocessing.sequence.pad_sequences(
    train_data, # list of list
    value = word_index['<PAD>'],
    padding = 'post', # post, pre
    maxlen = max_length
)

test_data = keras.preprocessing.sequence.pad_sequences(
    test_data, # list of list
    value = word_index['<PAD>'],
    padding = 'post', # post, pre
    maxlen = max_length
)

print(train_data[0])

In [ ]:
embedding_dim = 16
batch_size = 128

single_rnn_model = keras.models.Sequential([
        # 1. define matrix: [vocab_size, embedding_dim]
        # 2. [1,2,3,4..], max_length * embedding_dim
        # 3. batch_size * max_length * embedding_dim
        keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        keras.layers.LSTM(units=64, return_sequences=False),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])
single_rnn_model.summary()
single_rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
epochs = 30
single_rnn_history = single_rnn_model.fit(train_data, train_labels,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_split=0.2)

In [ ]:
def plot_learning_curves(history, label, epochs, min_value, max_value):
    data = {}
    data[label] = history.history[label]
    data['val_' + label] = history.history['val_' + label]
    pd.DataFrame(data).plot(figsize=(8, 5))
    plt.grid(True)
    plt.axis([0, epochs, min_value, max_value])
    plt.show()

plot_learning_curves(single_rnn_history, 'accuracy', epochs, 0, 1)
plot_learning_curves(single_rnn_history, 'loss', epochs, 0, 1)

In [ ]:
single_rnn_model.evaluate(test_data, test_labels, batch_size=batch_size)

In [ ]:
model = keras.models.Sequential([
        # 1. define matrix: [vocab_size, embedding_dim]
        # 2. [1,2,3,4..], max_length * embedding_dim
        # 3. batch_size * max_length * embedding_dim
        keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        keras.layers.Bidirectional(keras.layers.LSTM(units=64, return_sequences=True)),
        keras.layers.Bidirectional(keras.layers.LSTM(units=64, return_sequences=False)),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_data, train_labels,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_split=0.2)
plot_learning_curves(history, 'accuracy', epochs, 0, 1)
plot_learning_curves(history, 'loss', epochs, 0, 1)
model.evaluate(test_data, test_labels, batch_size=batch_size)

In [ ]:
bi_rnn_model = keras.models.Sequential([
        # 1. define matrix: [vocab_size, embedding_dim]
        # 2. [1,2,3,4..], max_length * embedding_dim
        # 3. batch_size * max_length * embedding_dim
        keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        keras.layers.Bidirectional(keras.layers.LSTM(units=32, return_sequences=False)),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])
bi_rnn_model.summary()
bi_rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
bi_rnn_history = bi_rnn_model.fit(train_data, train_labels,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_split=0.2)
plot_learning_curves(bi_rnn_history, 'accuracy', epochs, 0, 1)
plot_learning_curves(bi_rnn_history, 'loss', epochs, 0, 1)
bi_rnn_model.evaluate(test_data, test_labels, batch_size=batch_size)